In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import regularizers

# Define constants
NUM_CLASSES = 4  # Number of classes (correct, partially correct, incorrect, none)
IMG_WIDTH, IMG_HEIGHT = 299, 299  # Input image dimensions for Xception
BATCH_SIZE = 32
EPOCHS = 10

# Load Xception pre-trained model (include_top=False to exclude the classification layers)
base_model = Xception(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

# Unfreeze the last 30 layers of the base model for fine-tuning
for layer in base_model.layers[-30:]:
    layer.trainable = True

# Add custom classification layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = Dropout(0.5)(x)  # Adjusted dropout rate
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Implement a learning rate schedule
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Create data generators for training and validation with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    "/home/adarsh/Desktop/yoga/train/",
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

validation_generator = validation_datagen.flow_from_directory(
    "/home/adarsh/Desktop/yoga/test/",
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Define early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with early stopping and learning rate schedule
model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // BATCH_SIZE,
    callbacks=[early_stop, lr_schedule]  # Add early stopping and LR schedule to the callbacks
)

# Save the model for later use
model.save('pose_classification_model.h5')


Found 24855 images belonging to 4 classes.
Found 5636 images belonging to 4 classes.
Epoch 1/10
776/776 [==============================] - 4020s 5s/step - loss: 0.5976 - accuracy: 0.9519 - val_loss: 0.0853 - val_accuracy: 0.9830 - lr: 0.0010
Epoch 2/10
776/776 [==============================] - 4047s 5s/step - loss: 0.0652 - accuracy: 0.9901 - val_loss: 0.3153 - val_accuracy: 0.9418 - lr: 0.0010
Epoch 3/10
776/776 [==============================] - 4056s 5s/step - loss: 0.1241 - accuracy: 0.9821 - val_loss: 0.0350 - val_accuracy: 0.9936 - lr: 0.0010
Epoch 4/10
776/776 [==============================] - 4061s 5s/step - loss: 0.0885 - accuracy: 0.9881 - val_loss: 0.0499 - val_accuracy: 0.9938 - lr: 0.0010
Epoch 5/10
776/776 [==============================] - 4042s 5s/step - loss: 0.0331 - accuracy: 0.9936 - val_loss: 0.0239 - val_accuracy: 0.9963 - lr: 0.0010
Epoch 6/10
776/776 [==============================] - 4028s 5s/step - loss: 0.0787 - accuracy: 0.9870 - val_loss: 0.0127 - val_acc

/home/adarsh/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
